# Test if the notebook server is authorized to use Earth Engine

  * Click on the (potentially minimized) code block below and then press the 'Run' button to test if this server is authorized to talk to the Earth Engine backend servers.
  * If the server is authorized, you should see a map.
  * If the server is not authorized, you will see instructions for authorizing it.

Check if the IPython Widgets library is available on the server.

In [ ]:
try:
  import ipywidgets
  print('The IPython Widgets library (version {0}) is available on this server.'.format(
      ipywidgets.__version__
    ))
except ImportError:
  print('The IPython Widgets library is not available on this server.\n'
        'Please see https://github.com/jupyter-widgets/ipywidgets '
        'for information on installing the library.')
  raise


Check if the Earth Engine API is available on the server.

In [ ]:
try:
  import ee
  print('The Earth Engine Python API (version {0}) is available on this server.'.format(
      ee.__version__
    ))
except ImportError:
  print('The Earth Engine Python API library is not available on this server.\n'
        'Please see https://developers.google.com/earth-engine/python_install '
        'for information on installing the library.')
  raise

In [ ]:
import urllib
import cStringIO

def handle_submit(sender):
  token = ee.oauth.request_token(get_auth_textbox.value.strip())
  ee.oauth.write_token(token)
  get_auth_response.value = 'Successfully saved authorization token.'
  get_auth_textbox.value = ''  # Clear the textbox.
  setAuthorizedStatus()

def isAuthorized():
  try:
    ee.Initialize()
    return True
  except:
    return False
  
def setAuthorizedStatus():
  if isAuthorized():
    auth_status_button.description = 'success!'
    auth_status_button.button_style = 'success'
  else:
    auth_status_button.description = 'Not authorized'
    auth_status_button.button_style = 'danger'
    
# Define layouts for the table.
form_layout = ipywidgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    border='solid',
    width='100%'
)
form_row_layout = ipywidgets.Layout(
    display='flex',
    flex_flow='row',
)
form_row_item_desc_layout = ipywidgets.Layout(width="100%")
form_row_item_status_layout = ipywidgets.Layout(width="100%")

# Define widgets that may be displayed.
#ee_library_status_button = ipywidgets.Button(layout=form_row_item_status_layout)
auth_status_button = ipywidgets.Button(layout=form_row_item_status_layout)

get_auth_textbox = ipywidgets.Text(
  placeholder='Paste authentication code here',
  description='Code:'
)
get_auth_textbox.on_submit(handle_submit)

get_auth_response = ipywidgets.Label('')

setAuthorizedStatus()

if isAuthorized():
  # Get a sample Earth Engine image.  
  url = ee.Image("CGIAR/SRTM90_V4").getThumbUrl({'min':0, 'max':3000})
  auth_instructions = ipywidgets.VBox(
    [ipywidgets.Image(
      value=cStringIO.StringIO(urllib.urlopen(url).read()).read(),
      format='png',
      width=400,
      height=200
    )],
    layout=form_row_item_status_layout
  )
else:
  auth_instructions = ipywidgets.VBox([
      ipywidgets.HTML(
        'Click <a href="{url}" target="auth">here</a> to start the authentication and authorization process.<br/>'
        'When you receive an authentication code, paste it in the textbox below.'.format(
          url=ee.oauth.get_authorization_url()
        )
      ),
      get_auth_textbox,
      get_auth_response
    ],
    layout=form_row_item_status_layout
  )

form_items = [
  ipywidgets.Box(
    [
      ipywidgets.Label(value='Test Earth Engine authorization:', layout=form_row_item_desc_layout),
      auth_status_button
    ],
    layout=form_row_layout
  ),
  ipywidgets.Box(
    [
      ipywidgets.Label(value='', layout=form_row_item_desc_layout),
      auth_instructions
    ],
    layout=form_row_layout
  )
]

form = ipywidgets.Box(form_items, layout=form_layout)
form